# Genetic Improvement Demo

### Create the initial population of Genomes

In [1]:
# system imports
import os, sys
from tqdm import tqdm 


# set the project root directory (~/projects/GIMC)
project_root = os.path.expanduser("~/projects/GIMC")

# add project root to sys.path
sys.path.append(project_root)

# project imports
from genetic_improvement.ollamachat import OllamaChat
from genetic_improvement.config import USER_PROMPT, UNIT_TEST_CODE, SYSTEM_PROMPT, NUM_VARIANTS, BSI_CLASSIFICATION
from genetic_improvement.genome import Genome

# CONSTS
POPULATION_SIZE = 8


In [2]:
# get the initial population from the LLM
population = []
for i in tqdm(range(POPULATION_SIZE // NUM_VARIANTS)):
    ollama_chat = OllamaChat(model="qwen3-coder:latest", system_prompt=SYSTEM_PROMPT, temperature=0.3)
    chat_response = ollama_chat.chat(user_text=USER_PROMPT, stream=False)

    # parse the chat response and add it to the population
    variants = OllamaChat.parse_variants(chat_response)
    candidates = OllamaChat.submit_variants(variants, classification=BSI_CLASSIFICATION)
    for candidate in candidates:
        if candidate is not None:
            genome = Genome(candidate_hash=candidate)
            population.append(genome)

 50%|█████     | 1/2 [00:41<00:41, 41.04s/it]

Code submitted successfully:  0795675eeb9597807543583e8e5b9b0264604dde9cedd56c77b6388ebc9126a1
Code submitted successfully:  36ec8914e0b65c1759a9cd9064ac818a41302e84eabaf868b87d0968f6ecf3d4
Code submitted successfully:  0c7de6ac400a1ec1db5aed385f804d8ec9f1eb8dbec977dca4a3aedfad3b28b3
Code submitted successfully:  df557d1620a4cf49d4a0699494a155eeb97a7332becb5c0f6a70ad4c5d62c112


100%|██████████| 2/2 [01:25<00:00, 42.60s/it]

Code submitted successfully:  bcf2c8e5bd989284981b3f20bc974d57f89bff863460773a432fffd34a23625e
Code submitted successfully:  5ccf5379779bc13ce58b02faaf65a41e4a904f441dfcf3391cbe7e9da1bdf837
Code submitted successfully:  adef7b8ffd57b9ded0cd4ba786788477023fac7ab283ce91d15b053b069a575e
Code submitted successfully:  e449d0dd179ce5d5986b8921afcb6c6ed88781909d0bd2694dcdac3a1869a158


In [4]:
for genome in population:
    print(genome.get_candidate())

Candidate ID: 0795675eeb9597807543583e8e5b9b0264604dde9cedd56c77b6388ebc9126a1, Classification: com, Status: 3, Fitness: 8.933e-02, F1: 2.500e-01, F2: 3.333e-01, F3: 8.109e-05, Analysis ID: 17950, Error: None
Candidate ID: 36ec8914e0b65c1759a9cd9064ac818a41302e84eabaf868b87d0968f6ecf3d4, Classification: com, Status: 3, Fitness: 8.242e-03, F1: 7.692e-02, F2: 0.000e+00, F3: 0.000e+00, Analysis ID: None, Error: Compilation failed: 4 errors, 0 warnings
Candidate ID: 0c7de6ac400a1ec1db5aed385f804d8ec9f1eb8dbec977dca4a3aedfad3b28b3, Classification: com, Status: 3, Fitness: 1.177e-03, F1: 1.099e-02, F2: 0.000e+00, F3: 0.000e+00, Analysis ID: None, Error: Compilation failed: 30 errors, 0 warnings
Candidate ID: df557d1620a4cf49d4a0699494a155eeb97a7332becb5c0f6a70ad4c5d62c112, Classification: com, Status: 3, Fitness: 5.639e-03, F1: 5.263e-02, F2: 0.000e+00, F3: 0.000e+00, Analysis ID: None, Error: Compilation failed: 6 errors, 0 warnings
Candidate ID: bcf2c8e5bd989284981b3f20bc974d57f89bff863460

## Apply Edits to Genomes

In [8]:
# get the best genome from the population
best_genome = max(population, key=lambda g: g.get_candidate().get_fitness())
print("Best genome:", best_genome.get_candidate(), "Fitness:", best_genome.get_candidate().get_fitness())

# sort the population by fitness
population.sort(key=lambda g: g.get_candidate().get_fitness(), reverse=True)
print("Sorted population:")
for genome in population:
    print(genome.get_candidate())

Best genome: Candidate ID: 0795675eeb9597807543583e8e5b9b0264604dde9cedd56c77b6388ebc9126a1, Classification: com, Status: 3, Fitness: 8.933e-02, F1: 2.500e-01, F2: 3.333e-01, F3: 8.109e-05, Analysis ID: 17950, Error: None Fitness: 0.0893267422229439
Sorted population:
Candidate ID: 0795675eeb9597807543583e8e5b9b0264604dde9cedd56c77b6388ebc9126a1, Classification: com, Status: 3, Fitness: 8.933e-02, F1: 2.500e-01, F2: 3.333e-01, F3: 8.109e-05, Analysis ID: 17950, Error: None
Candidate ID: bcf2c8e5bd989284981b3f20bc974d57f89bff863460773a432fffd34a23625e, Classification: com, Status: 3, Fitness: 8.933e-02, F1: 2.500e-01, F2: 3.333e-01, F3: 8.109e-05, Analysis ID: 17951, Error: None
Candidate ID: e449d0dd179ce5d5986b8921afcb6c6ed88781909d0bd2694dcdac3a1869a158, Classification: com, Status: 3, Fitness: 1.531e-02, F1: 1.429e-01, F2: 0.000e+00, F3: 0.000e+00, Analysis ID: None, Error: Compilation failed: 2 errors, 0 warnings
Candidate ID: 36ec8914e0b65c1759a9cd9064ac818a41302e84eabaf868b87d096